# Generate surfaces from mask(s)

We will use the [ATAG atlas](https://www.nitrc.org/projects/atag) and generate a surface for each region

In [1]:
!sudo apt update && sudo apt install -y libgl1
!pip install nibabel pandas vtk

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [109 kB]      
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]    
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [101 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1,275 kB]    
Get:7 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [177 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [21.6 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [651 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]0m
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [934 kB]3m
Get:13 http://archive.ubuntu

7Progress: [ 10%] [#####.....................................................] 8Selecting previously unselected package libdrm-intel1:amd64.
Preparing to unpack .../06-libdrm-intel1_2.4.102-1ubuntu1~20.04.1_amd64.deb ...
7Progress: [ 10%] [######....................................................] 8Unpacking libdrm-intel1:amd64 (2.4.102-1ubuntu1~20.04.1) ...
7Progress: [ 11%] [######....................................................] 8Selecting previously unselected package libdrm-nouveau2:amd64.
Preparing to unpack .../07-libdrm-nouveau2_2.4.102-1ubuntu1~20.04.1_amd64.deb ...
7Progress: [ 12%] [######....................................................] 8Unpacking libdrm-nouveau2:amd64 (2.4.102-1ubuntu1~20.04.1) ...
Selecting previously unselected package libdrm-radeon1:amd64.
Preparing to unpack .../08-libdrm-radeon1_2.4.102-1ubuntu1~20.04.1_amd64.deb ...
7Progress: [ 14%] [#######...................................................] 8Unpacking libdrm-radeon1:amd64 (2.4.1

7Progress: [ 46%] [##########################................................] 8Selecting previously unselected package libglx0:amd64.
Preparing to unpack .../29-libglx0_1.3.2-1~ubuntu0.20.04.1_amd64.deb ...
7Progress: [ 47%] [###########################...............................] 8Unpacking libglx0:amd64 (1.3.2-1~ubuntu0.20.04.1) ...
7Progress: [ 48%] [###########################...............................] 8Selecting previously unselected package libgl1:amd64.
Preparing to unpack .../30-libgl1_1.3.2-1~ubuntu0.20.04.1_amd64.deb ...
Unpacking libgl1:amd64 (1.3.2-1~ubuntu0.20.04.1) ...
7Progress: [ 50%] [############################..............................] 8Setting up libxcb-dri3-0:amd64 (1.14-2) ...
7Progress: [ 50%] [#############################.............................] 87Progress: [ 51%] [#############################.............................] 8Setting up libx11-xcb1:amd64 (2:1.6.9-2ubuntu1.1) ...
7Progress: [ 52%] [#############################

In [5]:
import os
import requests
import zipfile
import nibabel as nib
import numpy as np
import vtk
from vtk.util import numpy_support

In [2]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

## Download zip with volumes

In [3]:
atag_url = 'https://www.nitrc.org/frs/download.php/9753/Final_BSAF_2017_ATAG_prop_masks.zip'
filename = download_file(atag_url)

## Unzip downloaded file

In [6]:
zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall('ATAG')
zip_ref.close()

## Generate surfaces and parcellation volume

In [14]:
def generate_suface(nifti_filename, output_filename=None):
    if output_filename is None:
        output_filename = nifti_filename.split('.')[0] + '.vtp'
        
    # Load nifti mask
    img = nib.load(nifti_filename)
    img_data = img.get_fdata()
    affine = img.affine

    # Generate mask volume
    volume = np.zeros(img_data.shape, dtype=int)
    volume[img_data>0] = 1

    # Generate padded volume to get closed surfaces
    padded_volume = np.zeros([s+2 for s in volume.shape], dtype=int)
    padded_volume[1:-1, 1:-1, 1:-1] = volume

    vtk_array = numpy_support.numpy_to_vtk(padded_volume.swapaxes(0, 2).ravel(), deep=True, array_type=vtk.VTK_INT)

    # Build VTK image data with the mask
    vtk_volume = vtk.vtkImageData()
    vtk_volume.SetDimensions(*padded_volume.shape)
    vtk_volume.SetSpacing([1, 1, 1])
    vtk_volume.AllocateScalars(vtk.VTK_INT, 1)
    vtk_volume.GetPointData().SetScalars(vtk_array)

    # Generate mesh using marching cubes
    marching_cubes = vtk.vtkDiscreteMarchingCubes()
    marching_cubes.SetInputData(vtk_volume)
    marching_cubes.GenerateValues(1, 0, 1)

    # Smooth the generated mesh
    smoother = vtk.vtkWindowedSincPolyDataFilter()
    smoother.SetInputConnection(marching_cubes.GetOutputPort())
    smoother.SetNumberOfIterations(15)
    smoother.BoundarySmoothingOff()
    smoother.FeatureEdgeSmoothingOn()
    smoother.SetFeatureAngle(150.0)
    smoother.SetPassBand(0.005)
    smoother.NonManifoldSmoothingOn()
    smoother.NormalizeCoordinatesOn()
    smoother.Update()

    transform_matrix = vtk.vtkMatrix4x4()
    [[transform_matrix.SetElement(i, j, affine[i,j]) for i in range(4)] for j in range(4)]

    transform = vtk.vtkTransform()
    transform.SetMatrix(transform_matrix)

    transform_filter = vtk.vtkTransformPolyDataFilter()
    transform_filter.SetInputConnection(smoother.GetOutputPort())
    transform_filter.SetTransform(transform)

    # Write mesh to disk
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_filename)
    writer.SetInputConnection(transform_filter.GetOutputPort())
    writer.Write()

In [15]:
volume_array = None
affine = None
label = 1
labels = []
for root, dirs, files in os.walk(os.path.join('ATAG', 'MNI05')):
        for name in files:
            if 'age' in name and 'non' not in name and not 'vtp' in name: # use middle age and linear only
                print(name)
                
                # Generate surface
                generate_suface(os.path.join(root, name))
                
                # Append mask
                if volume_array is None:
                    volume = nib.load(os.path.join(root, name))
                    affine = volume.affine
                    volume_array = volume.get_fdata()
                    volume_array[volume_array>0.0] = label
                else:
                    volume = nib.load(os.path.join(root, name))
                    new_volume_array = volume.get_fdata()
                    volume_array[new_volume_array>0.0] = label
                label += 1
                labels.append(name)
                
volume = nib.Nifti1Image(volume_array, affine)
nib.save(volume, os.path.join('ATAG', 'MNI05', 'atag.nii.gz'))
print(labels)
                

GPe_L_prob_mni_linear_middle_age.nii.gz
GPe_R_prob_mni_linear_middle_age.nii.gz
GPi_L_prob_mni_linear_middle_age.nii.gz
GPi_R_prob_mni_linear_middle_age.nii.gz
PAG_prob_mni_linear_middle_age.nii.gz
RN_L_prob_mni_linear_middle_age.nii.gz
RN_R_prob_mni_linear_middle_age.nii.gz
SN_L_prob_mni_linear_middle_age.nii.gz
SN_R_prob_mni_linear_middle_age.nii.gz
STN_L_prob_mni_linear_middle_age.nii.gz
STN_R_prob_mni_linear_middle_age.nii.gz
STR_L_prob_mni_linear_middle_age.nii.gz
STR_R_prob_mni_linear_middle_age.nii.gz


In [19]:

affine = None
for root, dirs, files in os.walk(os.path.join('ATAG', 'MNI05')):
    for name in files:
        if 'age' in name and 'non' not in name and not 'vtp' in name: # use middle age and linear only
            if volume_array is None:
                volume = nib.load(os.path.join(root, name))
                affine = volume.affine
                volume_array = volume.get_fdata()
                volume_array[volume_array>0.0] = label
            else:
                volume = nib.load(os.path.join(root, name))
                new_volume_array = volume.get_fdata()
                volume_array[new_volume_array>0.0] = label
            label += 1.0
            labels.append(name)
                
volume = nib.Nifti1Image(volume_array, affine)
nib.save(volume, os.path.join('ATAG', 'MNI05', 'atag.nii.gz'))
print(labels)

['GPe_L_prob_mni_linear_middle_age.nii.gz', 'GPe_R_prob_mni_linear_middle_age.nii.gz', 'GPi_L_prob_mni_linear_middle_age.nii.gz', 'GPi_R_prob_mni_linear_middle_age.nii.gz', 'PAG_prob_mni_linear_middle_age.nii.gz', 'RN_L_prob_mni_linear_middle_age.nii.gz', 'RN_R_prob_mni_linear_middle_age.nii.gz', 'SN_L_prob_mni_linear_middle_age.nii.gz', 'SN_R_prob_mni_linear_middle_age.nii.gz', 'STN_L_prob_mni_linear_middle_age.nii.gz', 'STN_R_prob_mni_linear_middle_age.nii.gz', 'STR_L_prob_mni_linear_middle_age.nii.gz', 'STR_R_prob_mni_linear_middle_age.nii.gz']
